In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import random
import sys
import os
from optparse import OptionParser
import numpy as np
from torch import optim
from PIL import Image
from torch.autograd import Function, Variable
import matplotlib.pyplot as plt
import matplotlib 
from torchvision import transforms
from torch.utils.data import Dataset
import cv2
import glob
import pickle
from tqdm import tqdm
%matplotlib inline

In [17]:
k = tf.constant([
    [1, 0, 1],
    [2, 1, 0],
    [0, 0, 1]
], dtype=tf.float32, name='k')
i = tf.constant([
    [4, 3, 1, 0],
    [2, 1, 0, 1],
    [1, 2, 4, 1],
    [3, 1, 0, 2]
], dtype=tf.float32, name='i')
kernel = tf.reshape(k, [3, 3, 1, 1], name='kernel')
image  = tf.reshape(i, [1, 4, 4, 1], name='image')
res = tf.squeeze(tf.nn.conv2d(image, kernel, [1, 1, 1, 1], "VALID"))
with tf.Session() as sess:
    print(sess.run(res))

[[14.  6.]
 [ 6. 12.]]


In [84]:
k = torch.tensor([[1, 2,5, 3,4],[5,6,10,7,8]])#,[9,10,11,12],[13,14,15,16]])
k = torch.unsqueeze(k,0);
k = torch.unsqueeze(k,0);
k = k.type(torch.FloatTensor)
x = nn.functional.interpolate(k, size=None, scale_factor=0.5, mode='bilinear', align_corners=True)
with tf.Session() as sess:
    print(x)

tensor([[[[1., 4.]]]])


In [23]:
inputs = torch.tensor([[[1, 2, 3],[5,6,7],[8,9,10]],[[2, 2, 3],[5,6,7],[8,9,10]]])
inputs= torch.unsqueeze(inputs,0);
inputs = inputs.type(torch.FloatTensor)
weights = torch.tensor([[[[1, 0],[0,0]],[[3, 0],[0,0]],[[5, 0],[0,0]],[[3, 0],[0,0]]],[[[1, 0],[0,0]],[[3, 0],[0,0]],[[1, 0],[0,0]],[[3, 0],[0,0]]]])
weights = weights.type(torch.FloatTensor)
print(weights.shape)
res = nn.functional.conv_transpose2d (inputs, weights, stride=2)
print(res)
kernel = torch.tensor([[1/4, 1/2, 1/4],[1/2,1,1/2],[1/4,1/2,1/4]])
kernel = torch.unsqueeze(kernel,0);
kernel= torch.unsqueeze(kernel,0);
kernel = kernel.type(torch.FloatTensor)
res1 = nn.functional.conv_transpose2d (res, kernel)
a = res1.shape
res1 = res1[:,:,1:a[2]-2,1:a[3]-2]
with tf.Session() as sess:
    print(inputs)
    print(weights)
    print(res.shape)
    print(res)
    print(res1)
    print(res1.shape)
    print(a[3]-2)

torch.Size([2, 4, 2, 2])
tensor([[[[ 3.,  0.,  4.,  0.,  6.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [10.,  0., 12.,  0., 14.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [16.,  0., 18.,  0., 20.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.]],

         [[ 9.,  0., 12.,  0., 18.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [30.,  0., 36.,  0., 42.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [48.,  0., 54.,  0., 60.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.]],

         [[ 7.,  0., 12.,  0., 18.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [30.,  0., 36.,  0., 42.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [48.,  0., 54.,  0., 60.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.]],

         [[ 9.,  0., 12.,  0., 18.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [30.,  0., 36.,  0., 42.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.],
          [48.,  0., 54.,  0., 60.,  0.],
   

RuntimeError: Given transposed=1, weight of size [1, 1, 3, 3], expected input[1, 4, 6, 6] to have 1 channels, but got 4 channels instead